# Simplifying with Openie6

The Openie6 software takes as input a possibly complex or compound sentence X,
and it returns a list of simple sentences that contain all the
information in the original sentence X.

Anton Alekseev (AA) and Anastasia Predelina (AP) wrote a jupyter notebook
that installs and runs the code in the Openie6 repo https://github.com/alexeyev/openie6
An exact copy of notebook by  AA/AP is included in this folder. It is also publicly available at AA's google drive
under the URL

 https://colab.research.google.com/drive/1samvO-SH6Xgjf9ItlhAF1EmBZo5grBQb?usp=sharing



This notebook adds new code to the end of the AA/AP notebook. The purpose of the
new code is
to simplify short stories and movie scripts.

In [ ]:
# decide whether to use GPU or not at the beginning
NB_WITH_GPU = True

# IGL-CA: inference pipeline
Coordination analysis inference using the OpenIE6 model.

* Anton's [OpenIE6 fork](https://github.com/alexeyev/openie6)
* [OpenIE6 original repo](https://github.com/dair-iitd/openie6)
* [OpenIE6 original paper](https://aclanthology.org/2020.emnlp-main.306/)

Prepared by [Anton Alekseev](https://github.com/alexeyev) and [Anastasia Predelina](https://github.com/PredelinaAsya). Quite a bit of effort, tbh.

**NOTA BENE**: GPU environment should be enabled before running the code! If not possible, another code cell for CPU-only environment is available at the very end of the notebook.

## Getting the OpenIE6 code

In [ ]:
# ! rmdir openie6

In [ ]:
! git clone https://github.com/alexeyev/openie6.git

Cloning into 'openie6'...
remote: Enumerating objects: 1882, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1882 (delta 40), reused 26 (delta 18), pack-reused 1820
Receiving objects: 100% (1882/1882), 8.52 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (387/387), done.


In [ ]:
! mkdir openie6/models

## Downloading the model

Let's get the CA model trained by the paper authors. Re-uploaded it to my gdrive to reduce the downloading time.

In [ ]:
nb = 'amazon'
# nb = 'google'
if nb == 'google':  # colab
    # copies from Anton's google drive to /content/conj_model.zip, 3.61 Gb
    ! gdown 185DK6eWKT6j6Oes04OIVFwJOKri8EVXV
elif nb == 'amazon':  # sagemaker studio lab
    # copies from rrtucci google drive to ./conj_model.zip
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JAyUs9jvRrpFgw6W8pl2uHVoJPxugEf4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JAyUs9jvRrpFgw6W8pl2uHVoJPxugEf4" -O conj_model.zip && rm -rf /tmp/cookies.txt

--2023-06-16 04:01:26--  https://docs.google.com/uc?export=download&confirm=t&id=1JAyUs9jvRrpFgw6W8pl2uHVoJPxugEf4
Resolving docs.google.com (docs.google.com)... 142.251.12.102, 142.251.12.138, 142.251.12.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e9lsl4a06nfv33cb9342rusfic506gi9/1686888075000/06804474810563551288/*/1JAyUs9jvRrpFgw6W8pl2uHVoJPxugEf4?e=download&uuid=876c09b5-0de7-4fd3-b804-970fa2fa6eaa [following]
--2023-06-16 04:01:26--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e9lsl4a06nfv33cb9342rusfic506gi9/1686888075000/06804474810563551288/*/1JAyUs9jvRrpFgw6W8pl2uHVoJPxugEf4?e=download&uuid=876c09b5-0de7-4fd3-b804-970fa2fa6eaa
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 172.217.194.132,

In [ ]:
! unzip conj_model.zip

Archive:  conj_model.zip
   creating: conj_model/
  inflating: conj_model/epoch=28_eval_acc=0.854.ckpt  
   creating: conj_model/logs/
   creating: conj_model/logs/test/
  inflating: conj_model/logs/test.txt  
   creating: conj_model/logs/test/checkpoints/
  inflating: conj_model/logs/test/events.out.tfevents.1601263425.vsky017.5076.0  
  inflating: conj_model/logs/test/events.out.tfevents.1601263521.vsky017.5784.0  
  inflating: conj_model/logs/test/hparams.yaml  


In [ ]:
! rm conj_model.zip*
! mv conj_model* openie6/models/

## Creating a text sample

In [ ]:
ztz1 = 'The man, who had never liked the words' \
  ' "booby" and "boobyhatch,"' \
  ' and who liked them even less on a shining morning when there' \
  ' was a unicorn in the garden, thought for a moment.'
ztz2 = "I love Luciano Pavarotti and Jose Carreras."
ztz12 = ztz1 + "\n" + ztz2
print(ztz12)
with open("sentences.txt", "w") as f:
  f.write(ztz12)

The man, who had never liked the words "booby" and "boobyhatch," and who liked them even less on a shining morning when there was a unicorn in the garden, thought for a moment.
I love Luciano Pavarotti and Jose Carreras.


## Setting up Python3.6

In [ ]:
! sudo apt-get update -y -qq
! sudo apt-get install python3.6 python3.6-distutils python3.6-dev -qq

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpython3.6-minimal:amd64.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../0-libpython3.6-minimal_3.6.15-1+focal3_amd64.deb ...
Unpacking libpython3.6-minimal:amd64 (3.6.15-1+focal3) ...
Selecting previously unselected package python3.6-minimal.
Preparing to unpack .../1-python3.6-minimal_3.6.15-1+focal3_amd64.deb ...
Unpacking python3.6-minimal (3.6.15-1+focal3) ...
Selecting previously unselected package libpython3.6-stdlib:amd64.
Preparing to unpack .../

In [ ]:
! ls /usr/bin/python3*

/usr/bin/python3	    /usr/bin/python3.6m
/usr/bin/python3.10	    /usr/bin/python3.6m-config
/usr/bin/python3.10-config  /usr/bin/python3.8
/usr/bin/python3.6	    /usr/bin/python3.8-config
/usr/bin/python3.6-config   /usr/bin/python3-config


Here you will have to select `python3.6` manually

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 3
!sudo update-alternatives --config python3

update-alternatives: using /usr/bin/python3.6 to provide /usr/bin/python3 (python3) in auto mode
There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.6    3         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.6    3         manual mode
  3            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 


## Setting up the dependencies

In [ ]:
! rm get-pip*

rm: cannot remove 'get-pip*': No such file or directory


In [ ]:
! python3 --version
! wget https://bootstrap.pypa.io/pip/3.6/get-pip.py
! python get-pip.py

Python 3.6.15
--2023-06-16 04:04:54--  https://bootstrap.pypa.io/pip/3.6/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2159061 (2.1M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.06M  --.-KB/s    in 0.008s  

2023-06-16 04:04:54 (247 MB/s) - ‘get-pip.py’ saved [2159061/2159061]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 30.9 MB/s            
     |████████████████████████████████| 952 kB 58.5 MB/s            


Satisfying a few more dependencies just in case

In [ ]:
! python -m pip install -q wheel setuptools pip --upgrade

In [ ]:
! python -m pip install -q -r openie6/requirements.txt

     |████████████████████████████████| 12.0 MB 10.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 776.8 MB 16 kB/s              
     |████████████████████████████████| 123 kB 71.9 MB/s            
     |████████████████████████████████| 104 kB 76.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 3.7 MB 66.3 MB/s            
     |████████████████████████████████| 156 kB 77.9 MB/s            
     |████████████████████████████████| 133 kB 80.5 MB/s            
     |████████████████████████████████| 82 kB 1.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 829 kB 31.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 90 kB 9.8 MB/s             
     |████████████████████████████████| 3.0 MB 68.1 MB/s            
     |████████████████████████████████| 58 kB 6.3 MB/s             
     |██

In [ ]:
! python -m nltk.downloader stopwords
! python -m nltk.downloader punkt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Colab-specific dependencies

In [ ]:
! python -m pip install -q ipykernel google-colab ipywidgets

     |████████████████████████████████| 121 kB 18.2 MB/s            
     |████████████████████████████████| 72 kB 1.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 123 kB 84.3 MB/s            
     |████████████████████████████████| 130 kB 67.7 MB/s            
     |████████████████████████████████| 427 kB 53.8 MB/s            
     |████████████████████████████████| 64 kB 1.8 MB/s             
     |████████████████████████████████| 104 kB 11.0 MB/s            
     |████████████████████████████████| 758 kB 40.3 MB/s            
     |████████████████████████████████| 8.0 MB 25.1 MB/s            
     |████████████████████████████████| 10.1 MB 68.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 57 kB 6.1 MB/s             
     |████████████████████████████████| 484 kB 82.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 246 kB 75.8

## Running coordination analysis model

Please don't pay attention to the `UnknownBackend` warnings.

The `sentences.txt` samples will be converted to shorter sentences in `predictions.txt.conj`.

### GPU version (20 seconds or so)

In [ ]:
if NB_WITH_GPU:
  ! cd openie6 && CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0 PYTHONPATH=imojie:imojie/allennlp:imojie/pytorch_transformers:$PYTHONPATH python run.py --save models/conj_model --mode predict --inp ../sentences.txt --batch_size 1 --model_str bert-large-cased --task conj --gpus 1 --out ../predictions.txt

/usr/local/lib/python3.6/dist-packages/traitlets/config/configurable.py:84: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  self.config = config
[TerminalIPythonApp] WARNING | GUI event loop or pylab initialization failed

UnknownBackendTraceback (most recent call last)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py in enable_matplotlib(self, gui)
   2953         # Now we must activate the gui pylab wants to use, and fix %run to take
   2954         # plot updates into account
-> 2955         self.enable_gui(gui)
   2956         self.magics_manager.registry['ExecutionMagics'].default_runner = \
   2957             pt.mpl_runner(self.safe_execfile)

/usr/local/lib/python3.6/dist-packages/IPython/terminal/interactiveshell.py in enable_gui(self, gui)
    512         if gui:
    513             self.active_eventloop, self._inputhook =\
--> 514                 get_inputhook_name_and_func(gui)
    515         else:
    516             self.ac

Finally, the results.

In [ ]:
if NB_WITH_GPU:
  ! cat predictions.txt.conj

The man , who had never liked the words `` booby '' and `` boobyhatch , '' and who liked them even less on a shining morning when there was a unicorn in the garden , thought for a moment .
The man , '' , thought for a moment .
The man , who liked them even less on a shining morning when there was a unicorn in the garden , thought for a moment .
The man , who had never liked the words `` booby , thought for a moment .
The man , who had never liked the words `` boobyhatch , thought for a moment .

I love Luciano Pavarotti and Jose Carreras .
I love Luciano Pavarotti .
I love Jose Carreras .



### CPU version (40 seconds or so)

In [ ]:
if not NB_WITH_GPU:
  ! cd openie6 && PYTHONPATH=imojie:imojie/allennlp:imojie/pytorch_transformers:$PYTHONPATH python run.py --save models/conj_model --mode predict --inp ../sentences.txt --batch_size 1 --model_str bert-large-cased --task conj --out ../predictions.txt

In [ ]:
if not NB_WITH_GPU:
  ! cat predictions.txt.conj

## Application of this software in Mappa Mundi

In [ ]:
! git clone https://github.com/rrtucci/mappa_mundi.git

Cloning into 'mappa_mundi'...
remote: Enumerating objects: 687, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 687 (delta 112), reused 134 (delta 56), pack-reused 494
Receiving objects: 100% (687/687), 5.72 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (408/408), done.


In [ ]:
%ls

get-pip.py    openie6/              sample_data/
mappa_mundi/  predictions.txt.conj  sentences.txt


In [ ]:
reload_it = False
if reload_it:
  import importlib as ilib
  import simp_openie6
  ilib.reload(simp_openie6)

In [ ]:
import sys
sys.path.insert(0,'/content/mappa_mundi')
from my_globals import *
from utils import my_listdir
from simp_openie6 import openie6_simplify_batch_of_m_scripts
print(ZTZ_SIMPLIFIER)

simp_openie6


In [ ]:
USE_GPU = NB_WITH_GPU

def main3():
    print("************ simplifier:", ZTZ_SIMPLIFIER)
    in_dir = "short_stories_spell"
    out_dir = "short_stories_simp"
    in_dir = 'mappa_mundi/' + in_dir
    batch_file_names = my_listdir(in_dir)[0:3]
    openie6_simplify_batch_of_m_scripts(
        in_dir, out_dir,
        batch_file_names,
        verbose=False)


def main4():
    print("************ simplifier:", ZTZ_SIMPLIFIER)
    remove_dialogs = False
    in_dir = SPELL_DIR if not remove_dialogs else SPELL_RD_DIR
    out_dir = SIMP_DIR if not remove_dialogs else SIMP_RD_DIR
    in_dir = 'mappa_mundi/' + in_dir
    batch_file_names = my_listdir(in_dir)[0:3]
    openie6_simplify_batch_of_m_scripts(
        in_dir, out_dir,
        batch_file_names)

In [ ]:
main3()

************ simplifier: simp_openie6


In [ ]:
main4()

************ simplifier: simp_openie6


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r m_scripts_simp.zip m_scripts_simp
!zip -r short_stories_simp.zip short_stories_simp

  adding: m_scripts_simp/ (stored 0%)
  adding: m_scripts_simp/up.txt (deflated 66%)
  adding: m_scripts_simp/toy-story.txt (deflated 67%)
  adding: m_scripts_simp/wall-e.txt (deflated 64%)
  adding: short_stories_simp/ (stored 0%)
  adding: short_stories_simp/wiltons-holiday.txt (deflated 68%)
  adding: short_stories_simp/bill-the-bloodhound.txt (deflated 67%)
  adding: short_stories_simp/extricating-young-gussie.txt (deflated 69%)
